# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/video_games_sales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [1]:
import re
import pandas as pd
from IPython.display import display, HTML
from scipy import stats as sp
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
print("Исходные данные")
films = pd.read_csv("video_games_sales.csv")
films.head()

Исходные данные


,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


In [3]:
sports = films[films["Genre"] == "Sports"][["Critic_Score"]]
sports.dropna(inplace=True) # уберем строки без оценок

print("Будем считать, что критики хорошо относятся к спортивным играм, если оценка 5.0 и более.")
print("За нулевую гипотезу принимаем, что критики хорошо относятся к спортивнм играм.")

alpha = 0.05
print(f"Возьмем традиционный уровень значимости α = {alpha}.")
result = sp.ttest_1samp(sports["Critic_Score"], 5.0)
if (result.statistic < 0) & (result.pvalue / 2 < alpha):
    print("Отвергаем нулевую гипотезу, критики плохо относятся к спортивным играм.")
else:
    print("Не отвергаем нулевую гипотезу, критики хорошо относятся к спортивным играм.")

Будем считать, что критики хорошо относятся к спортивным играм, если оценка 5.0 и более.
За нулевую гипотезу принимаем, что критики хорошо относятся к спортивнм играм.
Возьмем традиционный уровень значимости α = 0.05.
Не отвергаем нулевую гипотезу, критики хорошо относятся к спортивным играм.


In [4]:
PC = films[films["Platform"] == "PC"][["Critic_Score"]]
PC.dropna(inplace=True) # уберем строки без оценок

PS4 = films[films["Platform"] == "PS4"][["Critic_Score"]]
PS4.dropna(inplace=True) # уберем строки без оценок

print("За нулевую гипотезу принимаем, что критики одинаково оценивают игры на платформах PC и PS4.")

alpha = 0.05
print(f"Возьмем традиционный уровень значимости α = {alpha}.")
result = sp.ttest_ind(PC["Critic_Score"], PS4["Critic_Score"], equal_var=False)
print(f"pvalue = {result.pvalue}.")
if result.pvalue < alpha:
    if result.statistic > 0:
        platform = "PC"
    else:
        platform = "PS4"
    print(f"Отвергаем нулевую гипотезу, критики лучше оценивают игры на платформе {platform}.")
else:
    print("Не отвергаем нулевую гипотезу, критики одинаково оценивают игры на платформах PC и PS4.")

За нулевую гипотезу принимаем, что критики одинаково оценивают игры на платформах PC и PS4.
Возьмем традиционный уровень значимости α = 0.05.
pvalue = 0.006931808250254211.
Отвергаем нулевую гипотезу, критики лучше оценивают игры на платформе PS4.


In [5]:
shooter = films[films["Genre"] == "Shooter"][["Critic_Score"]]
shooter.dropna(inplace=True) # уберем строки без оценок

strategy = films[films["Genre"] == "Strategy"][["Critic_Score"]]
strategy.dropna(inplace=True) # уберем строки без оценок

print("За нулевую гипотезу принимаем, что критики одинаково оценивают стрелялки и стратегии.")

alpha = 0.05
print(f"Возьмем традиционный уровень значимости α = {alpha}.")
result = sp.ttest_ind(shooter["Critic_Score"], strategy["Critic_Score"], equal_var=False)
print(f"p-value = {result.pvalue}.")
if result.pvalue < alpha:
    if result.statistic > 0:
        genre = "стрелялки"
    else:
        genre = "стратегии"
    print(f"Отвергаем нулевую гипотезу, критики лучше оценивают {genre}.")
else:
    print("Не отвергаем нулевую гипотезу, критики одинаково оценивают стрелялки и стратегии.")

За нулевую гипотезу принимаем, что критики одинаково оценивают стрелялки и стратегии.
Возьмем традиционный уровень значимости α = 0.05.
p-value = 0.10838786414223071.
Не отвергаем нулевую гипотезу, критики одинаково оценивают стрелялки и стратегии.


## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```python
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  
6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  
7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;  
8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);  
9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [6]:
def modify_message(col, sws, wnl):
    '''Модифицируем сообщение'''
    message = col.lower()
    message = re.sub('[\W_]+',' ', message)
    message = message.split()
    message = [word for word in message if word not in sws]
    message = [wnl.lemmatize(word) for word in message]
    message = " ".join(message)
    return message


# загружаем данные
spam = pd.read_csv("spam.csv")
spam_copy = spam.copy()
print(f"\nИсходные данные (первые 10 строк из {len(spam)})")
display(HTML(spam.head(10).to_html()))

# преобразовываем сообщения
spam["Message"] = spam["Message"].apply(modify_message, sws=set(stopwords.words('english')), wnl=WordNetLemmatizer())
print(f"\nДанные после преобразования сообщений (первые 10 строк из {len(spam)})")
display(HTML(spam.head(10).to_html()))

# создаем векторы TF-IDF
tfidf = TfidfVectorizer(use_idf=False, sublinear_tf=True) # небольшой тюнинг для улучшения качества предсказания
tfidf_matrix = tfidf.fit_transform(spam["Message"])
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names())
print(f"\nМатрица TF-IDF (первые 10 строк из {tfidf_matrix.shape[0]} и первые 10 столбцов из {tfidf_matrix.shape[1]})")
display(HTML(tfidf_matrix.iloc[:10, :10].to_html()))

# разбиваем данные на тренировочные и тестовые
replace = pd.factorize(spam["Category"])        # заменяем, хотя не обязательно
X_train, X_test = train_test_split(tfidf_matrix, test_size=0.3, random_state=42)
Y_train, Y_test = train_test_split(replace[0], test_size=0.3, random_state=42)

# строим модель логистической регрессии и получаем предсказания
model = LogisticRegression(random_state=42)
model.fit(X_train, Y_train)
model.score(X_train, Y_train)
Y_predict = model.predict(X_test)

# строим матрицу ошибок
cmtx = pd.DataFrame(confusion_matrix(Y_test, Y_predict), columns=replace[1], index=replace[1])
print("\nМатрица ошибок")
display(HTML(cmtx.to_html()))

# определяем качество предсказаний
accur = round(accuracy_score(Y_test, Y_predict) * 100, 2)
print(f"\nКачество предсказаний: {accur}%.")

# отбираем ошибочные значения
i_wrong = []                                     # список номеров строк с ошибочными значениями
Y_wrong = []                                     # список ошибочных значений
for i in range(len(Y_test)):
    if Y_test[i] != Y_predict[i]:
        i_wrong.append(X_test.index[i])          # берем индекс, который был в тестовой выборке
        Y_wrong.append(replace[1][Y_predict[i]]) # делаем обратную замену чисел на ham и spam

# формируем датафрейм с ошибочными предсказаниями
wrong = spam_copy.iloc[i_wrong].copy()
wrong["Category_predict"] = Y_wrong
wrong = wrong.sort_index()[["Category", "Category_predict", "Message"]]
print(f"\nОшибочные предсказания (первые 10 строк из {len(wrong)})")
display(HTML(wrong.head(10).to_html()))
print("Вывод: судя по качеству предсказаний модель получилась хорошая.")
print("При этом, модель гораздо чаще ошибалась считая спам не спамом, согласно матрице ошибок.")


Исходные данные (первые 10 строк из 5572)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030



Данные после преобразования сообщений (первые 10 строк из 5572)


,Category,Message
0,ham,go jurong point crazy available bugis n great world la e buffet cine got amore wat
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though
5,spam,freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv
6,ham,even brother like speak treat like aid patent
7,ham,per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune
8,spam,winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour
9,spam,mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030



Матрица TF-IDF (первые 10 строк из 5572 и первые 10 столбцов из 8048)


...


Матрица ошибок


,ham,spam
ham,1440,8
spam,48,176



Качество предсказаний: 96.65%.

Ошибочные предсказания (первые 10 строк из 56)


,Category,Category_predict,Message
19,spam,ham,"England v Macedonia - dont miss the goals/team news. Txt ur national team to 87077 eg ENGLAND to 87077 Try:WALES, SCOTLAND 4txt/ú1.20 POBOXox36504W45WQ 16+"
45,ham,spam,No calls..messages..missed calls
68,spam,ham,"Did you hear about the new ""Divorce Barbie""? It comes with all of Ken's stuff!"
75,ham,spam,I am waiting machan. Call me once you free.
135,spam,ham,Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p
227,spam,ham,"Will u meet ur dream partner soon? Is ur career off 2 a flyng start? 2 find out free, txt HORO followed by ur star sign, e. g. HORO ARIES"
305,spam,ham,"SMS. ac Blind Date 4U!: Rodds1 is 21/m from Aberdeen, United Kingdom. Check Him out http://img. sms. ac/W/icmb3cktz8r7!-4 no Blind Dates send HIDE"
333,spam,ham,Call Germany for only 1 pence per minute! Call from a fixed line via access number 0844 861 85 85. No prepayment. Direct access!
387,ham,spam,Customer place i will call you.
415,spam,ham,100 dating service cal;l 09064012103 box334sk38ch


Вывод: судя по качеству предсказаний модель получилась хорошая.
При этом, модель гораздо чаще ошибалась считая спам не спамом, согласно матрице ошибок.


#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
